In [1]:
import numpy as np
import scipy.linalg as spla
import scipy.stats as st
import math
import random as rd
import matplotlib.pyplot as plt

In [2]:
class def_event:
    def __init__(self,E,t):
        self.evnt = E
        self.time = t

class def_serial_episode:
    def __init__(self, evnts_ordr):
        self.evnt = evnts_ordr[:]
        self.freq = 0

class def_window:
    def __init__(self, st_time, end_time):
        self.ts = st_time
        self.te = end_time

In [3]:
class def_episode:
    def __init__(self, evnts, edge_set):
        self.freq = 0
        self.evnts = evnts.copy()
        self.edges = edge_set.copy()

class def_NFA:
    def __init__(self):
        self.s = set()
        self.S = list([self.s])
        self.F = -1
        self.D = {}

class def_DFA:
    def __init__(self):
        self.s = {0}
        self.S = list([self.s])
        self.F = list()
        self.D = {}

In [4]:
def MO_Span_Recursive(ep_p,ep_p_MO_list,Tx,FT):
    global F1, candidate_episodes
    F = list()
    closed = 1
    if len(ep_p.evnt) <= 5:
        for E in F1:
            if E not in ep_p.evnt:
                evnts_temp = ep_p.evnt.copy()
                evnts_temp.append(E)
                ep = def_serial_episode(evnts_temp)
                ep_MO_list = Compute_MO_Span_List(ep_p_MO_list,E)
                ep.freq = len(ep_MO_list)
                if ep.freq == ep_p.freq:
                    closed = 0
                if ep.freq >= FT:
                    F += MO_Span_Recursive(ep,ep_MO_list,Tx,FT)
                candidate_episodes.append(ep)
    F.append(ep_p)
    return(F)

In [5]:
def Compute_MO_Span_List(ep_p_MO_list,E):
    p_size = len(ep_p_MO_list)
    E_MO_list = MO_list[evnt_id[E]]
    E_size = len(E_MO_list)
    ep_MO_list = list()
    p1=0
    p2=0
    while (p1<p_size and p2<E_size):
        while p2 < E_size and E_MO_list[p2].ts <= ep_p_MO_list[p1].te :
            p2 += 1
        if p2 < E_size and E_MO_list[p2].te-ep_p_MO_list[p1].ts < Tx:
            while(p1 < p_size and ep_p_MO_list[p1].te < E_MO_list[p2].ts):
                p1 += 1
            ep_MO_list.append(def_window(ep_p_MO_list[p1-1].ts,E_MO_list[p2].te))
        else:
            p1 += 1
    return(ep_MO_list)

In [6]:
def Find_Closed(Freq_ep):
    hasher = list()
    hashtable = list()
    for ep in Freq_ep:
        fn = ep.freq
        flg = 1
        for k in range(len(hasher)):
            if fn == hasher[k]:
                flg = 0
                hashtable[k].append(ep)
                break
        if flg == 1:
            hasher.append(fn)
            hashtable.append(list([ep]))

    Freq_closed_ep = list()
    for k in range(len(hasher)):
        len_hash_k = len(hashtable[k])
        p = 0
        while p < len_hash_k:
            q = 0
            if q == p:
                q += 1
            while q < len_hash_k:
                len_p = len(hashtable[k][p].evnt)
                len_q = len(hashtable[k][q].evnt)
                if len_p < len_q:
                    sub_ep = 0
                    pi = 0
                    qi = 0
                    while(1):
                        if hashtable[k][p].evnt[pi] == hashtable[k][q].evnt[qi]:
                            pi += 1
                            if pi == len_p:
                                sub_ep = 1
                                break
                        qi += 1
                        if qi == len_q:
                            break
                    if sub_ep == 1:
                        hashtable[k].remove(hashtable[k][p])
                        len_hash_k = len(hashtable[k])
                        q -= 1
                        break
                q += 1
                if q == p:
                    q += 1
            if q >= len_hash_k:
                p += 1

    for k in range(len(hasher)):
        for l in range(len(hashtable[k])):
            Freq_closed_ep.append(hashtable[k][l])

    return Freq_closed_ep

In [7]:
def Construct_NFA(alpha):
    alpha_size = len(alpha.evnts)
    pi = {}
    for e in alpha.evnts:
        pi[e] = set()
    for (e1,e2) in alpha.edges:
        pi[e2].add(e1)
    NFA = def_NFA()
    n_states = 1
    for Q in NFA.S:
        NFA.D[str(Q)] = {}
        if len(Q) != alpha_size:
            for ev in alpha.evnts-Q:
                if not (alpha.evnts-Q).intersection(pi[ev]):
                    Q_new = Q.copy()
                    Q_new.add(ev)
                    if Q_new in NFA.S:
                        NFA.D[str(Q)][ev] = {NFA.S.index(Q_new)}
                    else:
                        NFA.S.append(Q_new)
                        NFA.D[str(Q)][ev] = {len(NFA.S)-1}
            NFA.D[str(Q)]['def'] = {NFA.S.index(Q)}
        else:
            NFA.D[str(Q)]['def'] = set()
            NFA.F = NFA.S.index(Q)
    for ev in alpha.evnts:
        if not alpha.evnts.intersection(pi[ev]):
            NFA.D[str(NFA.s)][ev].add(0)
    return NFA

In [8]:
def Construct_DFA(NFA):
    DFA = def_DFA()
    for state in DFA.S:
        if (NFA.F in state):
            DFA.F.append(state)
        DFA.D[str(state)] = {}
        W = set({})
        for substate in state:
            Q = str(NFA.S[substate])
            W = W.union(set(NFA.D[Q].keys()))
        for ev in W:
            new_state = set()
            for substate in state:
                Q = str(NFA.S[substate])
                if ev in NFA.D[Q].keys():
                    new_state = new_state.union(NFA.D[Q][ev])
                else:
                    new_state = new_state.union(NFA.D[Q]['def'])
            DFA.D[str(state)][ev] = new_state.copy()
            if new_state not in DFA.S:
                        DFA.S.append(new_state)
        keys = list(DFA.D[str(state)].keys())
        for k in range(len(keys)):
            if keys[k] != 'def' and DFA.D[str(state)][keys[k]] == DFA.D[str(state)]['def']:
                del DFA.D[str(state)][keys[k]]
            else:
                k += 1
    return DFA

In [9]:
def GetTransitionMatrix(DFA,prob):
    T0 = list()
    for state in DFA.S:
        T0.append([0]*len(DFA.S))
        for ev in DFA.D[str(state)].keys():
            if ev == 'def':
                continue
            T0[-1][DFA.S.index(DFA.D[str(state)][ev])] += prob[ev]
        T0[-1][DFA.S.index(DFA.D[str(state)]['def'])] = round(1 - sum(T0[-1]),5)
    return(T0)

In [10]:
def GetStats(T0,final_states):

    ndiags = len(T0)
    nfinals = len(final_states)
    
    if nfinals == 0:
        return(0,0)
    
    T = np.matrix(T0)
    L = np.eye(ndiags)

    dQdu = 0
    d2Qdu2 = 0
    for k1 in range(nfinals):
        L1 = L.copy()
        L1[final_states[k1],final_states[k1]] = 0
        dQdu += spla.det(L1-T)
        for k2 in range(k1+1,nfinals):
            L2 = L1.copy()
            L2[final_states[k2],final_states[k2]] = 0
            d2Qdu2 += spla.det(L2-T)
    d2Qdu2 *= 2

    dQdl = 0
    d2Qdl2 = 0
    d2Qdudl = 0
    for k1 in range(ndiags):
        T1 = T.copy()
        T1[:,k1] *= 0
        dQdl += spla.det(L-T1)
        for f1 in final_states:
            if k1 != f1:
                L1 = L.copy()
                L1[f1,f1] = 0
                d2Qdudl += spla.det(L1-T1)
        for k2 in range(k1+1,ndiags):
            T2 = T1.copy()
            T2[:,k2] *= 0
            d2Qdl2 += spla.det(L-T2)
    d2Qdl2 *= 2


    lambda1 = -dQdu/dQdl
    lambda11 = -(d2Qdu2 + 2*lambda1*d2Qdudl + lambda1**2*d2Qdl2)/dQdl

    MX1 = lambda1
    VX1 = lambda11 + lambda1 - lambda1**2

    return MX1, VX1

In [18]:
global candidate_episodes

evnt_strm = list()
print('Select one of the three datasets:')
print('1: Noisy data with no embedded episodes')
print('2: Noisy data with two 4-node episodes embedded')
print('3: Noisy data with two 6-node episodes embedded')
print('Enter your choice: (1/2/3): ')

while(1):
    option = int(input())

    if option == 1:
        name = './Data/Datastream_M_50_n_10000_N_4_emb_2_q1_1_q2_0.txt'
        Tx = 80
        FT = 50
        N = 0
        print('Dataset: ', name)
        print('No episodes are embedded')
        break
    elif option == 2:
        name = './Data/Datastream_M_50_n_10000_N_4_emb_2_q1_0.7_q2_0.txt'
        Tx = 50
        FT = 120
        N = 4
        print('Dataset: ', name)
        print('Episodes embedded: 1->2->3->4 and 5->6->7->8')
        break
    elif option == 3:
        name = './Data/Datastream_M_50_n_10000_N_6_emb_2_q1_0.7_q2_0.txt'
        Tx = 95
        FT = 100
        N = 6
        print('Dataset: ', name)
        print('Episodes embedded: 1->2->3->4->5->6 and 7->8->9->10->11->12')
        break
    else:
        print('Wrong option, please enter (1/2/3)')
print('Expiry Time (Tx): ', Tx)
print('Frequency Threshold: ', FT)

with open(name,'r') as f:
    for line in f:
        entry = line.split(',')
        evnt_strm.append(def_event(entry[0],int(entry[1][:-1])))

M = len(evnt_strm)
alph = list()
for m in range(len(evnt_strm)):
    A = evnt_strm[m].evnt
    flg = 1
    for a in alph:
        if a == A:
            flg = 0
            break
    if flg == 1:
        alph.append(A)
LA = len(alph) 

A = alph[0]
evnt_id = {A: 0}
count = 1
for A in alph[1:]:
    evnt_id[A] = count
    count += 1

MO_list = list()
for i in range(LA):
    MO_list.append(list())

Freq_ep = list()
F1 = alph
F1_ep = list()
for A in alph:
    F1_ep.append(def_serial_episode(list([A])))

for event in evnt_strm:
    E = event.evnt
    t = event.time
    F1_ep[evnt_id[E]].freq += 1
    MO_list[evnt_id[E]].append(def_window(t,t))

candidate_episodes = list([])
for E in F1:
    if F1_ep[evnt_id[E]].freq >= FT:
        Freq_ep += MO_Span_Recursive(F1_ep[evnt_id[E]],MO_list[evnt_id[E]],Tx,FT)

# print('The following are the frequent episodes discovered')
# for ep in Freq_ep:
#     print(ep.evnt ,':', ep.freq)
    
Freq_closed_ep = Find_Closed(Freq_ep)

print('Total closed frequent episodes: ', len(Freq_closed_ep))

count_CE_2 = 0
count_CE_3 = 0
count_CE_4 = 0
count_CE_5 = 0
count_CE_6 = 0
for k in range(len(candidate_episodes)):
    if len(candidate_episodes[k].evnt) == 2:
        count_CE_2 += 1
    if len(candidate_episodes[k].evnt) == 3:
        count_CE_3 += 1
    if len(candidate_episodes[k].evnt) == 4:
        count_CE_4 += 1
    if len(candidate_episodes[k].evnt) == 5:
        count_CE_5 += 1
    if len(candidate_episodes[k].evnt) == 6:
        count_CE_6 += 1

count_FE_2 = 0
count_FE_3 = 0
count_FE_4 = 0
count_FE_5 = 0
count_FE_6 = 0
for k in range(len(Freq_closed_ep)):
    if len(Freq_closed_ep[k].evnt) == 2:
        count_FE_2 += 1
    if len(Freq_closed_ep[k].evnt) == 3:
        count_FE_3 += 1
    if len(Freq_closed_ep[k].evnt) == 4:
        count_FE_4 += 1
    if len(Freq_closed_ep[k].evnt) == 5:
        count_FE_5 += 1
    if len(Freq_closed_ep[k].evnt) == 6:
        count_FE_6 += 1

# print(count_CE_2, count_FE_2)
# print(count_CE_3, count_FE_3)
# print(count_CE_4, count_FE_4)
# print(count_CE_5, count_FE_5)
# print(count_CE_6, count_FE_6)


Select one of the three datasets:
1: Noisy data with no embedded episodes
2: Noisy data with two 4-node episodes embedded
3: Noisy data with two 6-node episodes embedded
Enter your choice: (1/2/3): 


 3


Dataset:  ./Data/Datastream_M_50_n_10000_N_6_emb_2_q1_0.7_q2_0.txt
Episodes embedded: 1->2->3->4->5->6 and 7->8->9->10->11->12
Expiry Time (Tx):  95
Frequency Threshold:  100
Total closed frequent episodes:  26248


In [19]:
serial_episodes = list()
episodes = list()
for ep in Freq_closed_ep:
    if len(ep.evnt) >=2 and len(ep.evnt) <=6:
        serial_episodes.append(ep)
        
        temp_evnts = set(ep.evnt)
        temp_edges = set()
        for k in range(len(ep.evnt)):
            for l in range(k+1,len(ep.evnt)):
                temp_edges.add((ep.evnt[k],ep.evnt[l]))
        episodes.append(def_episode(temp_evnts,temp_edges))
#         print(ep.evnt ,':', ep.freq)

n = 10000
Prob_Ep_Events = {}
for E in F1:
    Prob_Ep_Events[E] = len(MO_list[evnt_id[E]])/n
    
mean_estimate = list()
var_estimate = list()

for episode_id in range(len(episodes)):
    NFA = Construct_NFA(episodes[episode_id])
    DFA = Construct_DFA(NFA)

    final_states = list()
    for k in range(len(DFA.S)):
        if (NFA.F in DFA.S[k]):
            final_states.append(k)

    T0 = GetTransitionMatrix(DFA,Prob_Ep_Events)
    MX1, VX1 = GetStats(T0,final_states)
    
    mean_estimate.append(MX1)
    var_estimate.append(VX1)

In [21]:
SSTh2 = list()
print('Enter the value of c: (0.01 is typical)')
c = float(input())
c = c/len(serial_episodes)
for k in range(len(serial_episodes)):
    value = mean_estimate[k]*n + math.sqrt(var_estimate[k]*n)*st.norm.ppf(1-c)
    SSTh2.append(value)

significant_episodes = list()

count_SS2_2 = 0
count_SS2_3 = 0
count_SS2_4 = 0
count_SS2_5 = 0
count_SS2_6 = 0
for k in range(len(serial_episodes)):
    if serial_episodes[k].freq > SSTh2[k]:
        significant_episodes.append(serial_episodes[k])
        if len(serial_episodes[k].evnt) == 2:
            count_SS2_2 += 1 
        if len(serial_episodes[k].evnt) == 3: 
            count_SS2_3 += 1 
        if len(serial_episodes[k].evnt) == 4:
            count_SS2_4 += 1
            # if N ==4:
                # print(serial_episodes[k].evnt)
        if len(serial_episodes[k].evnt) == 5:
            count_SS2_5 += 1
        if len(serial_episodes[k].evnt) == 6:
            count_SS2_6 += 1
            # if N == 6:
                # print(serial_episodes[k].evnt)
print("#Node \t #Candidates \t #Frequents \t #Significants") 
print('2\t', count_CE_2, '\t', count_FE_2, '\t', count_SS2_2)
print('3\t', count_CE_3, '\t', count_FE_3, '\t', count_SS2_3) 
print('4\t', count_CE_4, '\t', count_FE_4, '\t', count_SS2_4) 
print('5\t', count_CE_5, '\t', count_FE_5, '\t', count_SS2_5) 
print('6\t', count_CE_6, '\t', count_FE_6, '\t', count_SS2_6)

if N != 0:
    print('The following are the significant episodes: ')
    for ep in significant_episodes:
        if len(ep.evnt) == N:
            print(ep.evnt)

Enter the value of c: (0.01 is typical)


 0.01


#Node 	 #Candidates 	 #Frequents 	 #Significants
2	 2450 	 932 	 4
3	 45072 	 4932 	 107
4	 232556 	 11448 	 521
5	 526792 	 8816 	 203
6	 396720 	 70 	 5
The following are the significant episodes: 
['7', '8', '9', '10', '11', '12']
['1', '2', '3', '4', '5', '6']
['8', '9', '10', '11', '12', '2']
['1', '7', '8', '9', '10', '11']
['8', '1', '2', '3', '4', '5']
